In [13]:
# import modules
import json
import torch
import tokenization

import numpy as np

from transformer.model import GPTModel

In [14]:
# read configuration file
config = json.load(open('config.json'))
config

{'kor_vocab_length': 50000,
 'eng_vocab_length': 28998,
 'd_model': 768,
 'd_ff': 2048,
 'd_k': 64,
 'd_v': 64,
 'num_layers': 12,
 'num_heads': 8,
 'start_word': '[SOS]',
 'end_word': '[EOS]',
 'sep_word': '[SEP]',
 'cls_word': '[CLS]',
 'pad_word': '[PAD]',
 'mask_word': '[MASK]'}

In [15]:
# configure device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cpu')

In [16]:
# configure tokenizer
tokenizer = tokenization.FullTokenizer(
    vocab_file='vocab/kor_vocab.txt', do_lower_case=False)
tokenizer.tokenize('23일 업계에 따르면 이들 3사는 최근 열린 각 사 주주총회에서 올해 코로나19 여파 등으로 반도체·디스플레이 시장의 대외 불확실성이 높아질 것으로 전망했다.')

['23',
 '##일',
 '업계에',
 '따',
 '##르',
 '##면',
 '이들',
 '3',
 '##사는',
 '최근',
 '열린',
 '각',
 '사',
 '주주',
 '##총회에서',
 '올해',
 '코',
 '##로',
 '##나',
 '##19',
 '여',
 '##파',
 '등으로',
 '반도체',
 '·',
 '디스플레이',
 '시장의',
 '대외',
 '불',
 '##확실',
 '##성',
 '##이',
 '높',
 '##아',
 '##질',
 '것으로',
 '전망',
 '##했다',
 '.']

In [19]:
# define sample dataset
dataset = [
    '미성년자 열댓 명을 비롯해 여성 70여명을 대상으로 성 착취 동영상을 찍도록 하고 이를 텔레그램에서 유포한 이른바 텔레그램 n번방 사건에 대해 23일 정치권이 분노의 목소리를 내고 있다.',
    '피의자에 대한 검찰의 포토라인 관행은 지난해 조국 정국에서 폐지됐다. 신 최고의원은 이를 지적하며 "그때 포토라인 폐지가 수사기관 개혁이라고 주장했고, 인권 수사라고 주장했던 사람들은 이제 \'그게 그거랑 같냐\'를 들먹이며 그때 그 사람에 대한 수사와 지금 n번방 피의자나 박사에 대한 수사는 다르다고 할 것"이라고 주장하기도 했다.',
    '신 최고위원은 "피의자를 포토라인에 세우는 것을 금지한 게 2019년 10월"이라며 "인권보호수사규칙을 제정하자고 주장한 장관이 누구인가. 검찰이 누구에 대해 수사를 하다가 압박으로 포토라인이 폐지되었나. 실제, 포토라인 폐지로 바로 수혜를 입은 사람이 누구의 가족이냐"고 되물었다.']

token_length = 128

dec_inputs, dec_outputs = [], []
for data in dataset:
    token = [config['start_word']]
    token.extend(tokenizer.tokenize(data)[:token_length])
    token.append(config['end_word'])
    dec_input = token[:-1]
    dec_output = token[1:]
    while len(dec_input) < token_length + 1:
        dec_input.append(config['pad_word'])
    while len(dec_output) < token_length + 1:
        dec_output.append(config['pad_word'])

    dec_input = tokenizer.convert_tokens_to_ids(dec_input)
    dec_output = tokenizer.convert_tokens_to_ids(dec_output)

    dec_inputs.append(dec_input)
    dec_outputs.append(dec_output)

print(dec_inputs)
print(dec_outputs)

dec_inputs = torch.as_tensor(dec_inputs, dtype=torch.long).to(device)
dec_outputs = torch.as_tensor(dec_outputs, dtype=torch.long).to(device)

[[1, 47546, 198, 5489, 143, 3, 810, 3, 3873, 1533, 170, 7179, 36401, 47492, 47901, 416, 20249, 5101, 48189, 196, 7758, 135, 1918, 2082, 127, 14553, 10552, 47529, 275, 118, 40040, 222, 2082, 127, 14553, 47551, 609, 186, 40471, 40819, 2629, 147, 1418, 547, 104, 47567, 255, 162, 47709, 11060, 3089, 2769, 47440, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [1, 27041, 143, 469, 1514, 4479, 162, 2397, 4734, 47505, 304, 1333, 5904, 47507, 139, 47468, 139, 10552, 18664, 1143, 47440, 47523, 27428, 47265, 1918, 28825, 3005, 47452, 36660, 2397, 4734, 18664, 119, 3542, 1270, 2858, 22812, 47483, 213, 5889, 47453, 4924, 3542, 5721, 47483, 213, 4972, 5509, 13938, 10233, 47488, 47522, 348, 47522, 241, 23017, 3, 47488, 47462, 47493, 34865, 6182, 36660, 47522, 5509, 469, 1514, 3542, 1367, 5181, 47551, 609, 186, 27041, 143,

In [20]:
## configure model, optimizer, criterion
pad_index = tokenizer.convert_tokens_to_ids([config['pad_word']])[0]

generator = GPTModel(
    vocab_size=config['kor_vocab_length'],
    d_model=config['d_model'],
    d_ff=config['d_ff'],
    d_k=config['d_k'], d_v=config['d_v'],
    n_heads=config['num_heads'], n_layers=config['num_layers'],
    pad_index=pad_index, device=device).to(device)

optimizer = torch.optim.Adam(generator.parameters(), lr=5e-5)
criterion = torch.nn.CrossEntropyLoss()
generator

GPTModel(
  (decoder): MaskedDecoder(
    (tgt_emb): Embedding(50000, 768)
    (pos_emb): PositionalEncoding(
      (dropout): Dropout(p=0, inplace=False)
    )
    (layers): ModuleList(
      (0): MaskedDecoderLayer(
        (dec_self_attn): MultiHeadAttention(
          (WQ): Linear(in_features=768, out_features=512, bias=True)
          (WK): Linear(in_features=768, out_features=512, bias=True)
          (WV): Linear(in_features=768, out_features=512, bias=True)
          (linear): Linear(in_features=512, out_features=768, bias=True)
          (layer_norm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        )
        (pos_ffn): PoswiseFeedForwardNet(
          (l1): Linear(in_features=768, out_features=2048, bias=True)
          (l2): Linear(in_features=2048, out_features=768, bias=True)
          (relu): GELU()
          (layer_norm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        )
      )
      (1): MaskedDecoderLayer(
        (dec_self_attn): MultiHeadA

In [23]:
for epoch in range(100):

    optimizer.zero_grad()
    dec_logits, attns = generator(dec_inputs)
    loss = criterion(
        dec_logits.view(-1, dec_logits.size(-1)),
        dec_outputs.contiguous().view(-1))
    loss.backward()
    optimizer.step()

    print(epoch, loss.item())

0 10.976317405700684
1 7.627902507781982
2 6.312865257263184
3 6.122442722320557
4 5.987438201904297
5 5.792433261871338
6 5.655355930328369
7 5.556546211242676
8 5.4174957275390625
9 5.251761436462402
10 5.097161293029785
11 4.957839488983154
12 4.8264031410217285
13 4.694883346557617
14 4.557570457458496
15 4.413417339324951
16 4.265126705169678
17 4.115043640136719
18 3.963460922241211
19 3.8107032775878906
20 3.6577868461608887
21 3.5047521591186523
22 3.350825071334839
23 3.1961829662323
24 3.042105197906494
25 2.8892598152160645
26 2.7372820377349854
27 2.586233615875244
28 2.4370338916778564
29 2.2905523777008057
30 2.1473488807678223
31 2.0077075958251953
32 1.871577262878418
33 1.7393180131912231
34 1.6119086742401123
35 1.4898340702056885
36 1.373130202293396
37 1.2622491121292114
38 1.1575462818145752
39 1.0590561628341675
40 0.9672638773918152
41 0.8827014565467834
42 0.8052138686180115
43 0.734371542930603
44 0.6698440313339233
45 0.6113075613975525
46 0.5584662556648254
4

In [27]:
generator.eval()

eos_flag = tokenizer.convert_tokens_to_ids([config['end_word']])[0]
test_sentence = '신 최고위원은 "피의자를 포토라인에 세우는 것을 금지한 게 2019년'

tokens = [config['start_word']]
tokens.extend(tokenizer.tokenize(test_sentence))
test_sentence_dec = tokenizer.convert_tokens_to_ids(tokens)

while test_sentence_dec[-1] != eos_flag or len(test_sentence_dec) > 100:
    dec_inputs = torch.as_tensor([test_sentence_dec], dtype=torch.long).to(device)
    dec_logits, _ = generator(dec_inputs)
    predict = torch.argmax(dec_logits, axis=2)[:, -1].squeeze().detach().cpu().numpy()
    test_sentence_dec.append(int(predict))
    
print('----------------------')
print(f'input text : {tokens}')
predict_text = ' '.join(tokenizer.convert_ids_to_tokens(test_sentence_dec))
predict_text = predict_text.replace(" ##", "")
predict_text = predict_text.replace("##", "")
print(f'predict_text : {predict_text}')

----------------------
input text : ['[SOS]', '신', '최고', '##위원은', '"', '피의', '##자를', '포토', '##라인', '##에', '세', '##우', '##는', '것을', '금지', '##한', '게', '201', '##9', '##년']
predict_text : [SOS] 신 최고위원은 " 피의자를 포토라인에 세우는 것을 금지한 게 2019년 10월 " 이라며 " 인권보호수사규칙을 제정하자고 주장한 장관이 누구인가 . 검찰이 누구에 대해 수사를 하다가 압박으로 포토라인이 폐지되었나 . 실제 , 포토라인 폐지로 바로 수혜를 입은 사람이 누구의 [UNK] " 고 되물었다 . [EOS]
